In [8]:
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling_core.types.doc import ImageRefMode
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling_core.types.doc import PictureItem
import pdfplumber
from typing import List, Tuple, Any, Dict
import os
from glob import glob
from tqdm import tqdm


def _reconstruct_text_from_chars(chars, y_line_thresh=2.0, x_gap_factor=0.45):
    """
    Rebuild lines from pdfplumber chars.
    - y_line_thresh: ngưỡng gộp cùng 1 dòng theo |y_mid - y_mid_prev|
    - x_gap_factor:  hệ số * size trung bình để quyết định chèn khoảng trắng
    """
    if not chars:
        return ""

    for c in chars:
        c["y_mid"] = (c["top"] + c["bottom"]) / 2.0

    chars.sort(key=lambda c: (c["y_mid"], c["x0"]))

    lines = []
    cur = [chars[0]]
    for ch in chars[1:]:
        if abs(ch["y_mid"] - cur[-1]["y_mid"]) <= y_line_thresh:
            cur.append(ch)
        else:
            lines.append(cur)
            cur = [ch]
    lines.append(cur)

    out_lines = []
    for line in lines:
        line.sort(key=lambda c: c["x0"])

        sizes = [c.get("size", 10.0) for c in line]
        avg_size = sum(sizes) / len(sizes) if sizes else 10.0
        x_gap_thresh = avg_size * x_gap_factor
        buf = []
        for i, ch in enumerate(line):
            if i == 0:
                buf.append(ch["text"])
            else:
                prev = line[i-1]
                gap = ch["x0"] - prev["x1"]
                if gap > x_gap_thresh:
                    buf.append(" ")
                buf.append(ch["text"])
        s = "".join(buf).strip()
        if s:
            out_lines.append(s)

    return " ".join(out_lines).strip()

def refine_docling_texts_with_pdfplumber(
    pdf_path: str,
    docling_text_items: List[Any],   
) -> List[Tuple[int, str, str]]:
    """
    Duyệt toàn bộ items trong result.document.texts, dùng pdfplumber để refine text theo bbox.
    Trả về list (index, old_text, refined_text). Không sửa in-place để an toàn.
    """
    results = []

    with pdfplumber.open(pdf_path) as pdf:
        for i, item in enumerate(docling_text_items):
            try:

                if not getattr(item, "prov", None):
                    continue

                prov = None
                for p in item.prov:
                    if getattr(p, "bbox", None) and getattr(p, "page_no", None):
                        prov = p
                        break
                if prov is None:
                    continue
                page_no = int(prov.page_no)  
                bbox = prov.bbox             
                l, t, r, b = float(bbox.l), float(bbox.t), float(bbox.r), float(bbox.b)

                page = pdf.pages[page_no - 1]
                H = float(page.height)

                if getattr(page, "rotation", 0) in (90, 180, 270):
                    page = page.rotate(360 - page.rotation)
                    H = float(page.height)

                origin = str(getattr(bbox, "coord_origin", ""))
                if "BOTTOMLEFT" in origin:
                    plumber_bbox = (l, H - t, r, H - b)
                elif "TOPLEFT" in origin:
                    plumber_bbox = (l, t, r, b)
                else:
                    continue  

                cropped = page.crop(plumber_bbox)
                chars = cropped.chars
                if not chars:

                    results.append((i, getattr(item, "text", ""), getattr(item, "text", "")))
                    continue
                refined = _reconstruct_text_from_chars(chars)

                old_text = getattr(item, "text", "")
                results.append((i, old_text, refined))
            except Exception as e:

                results.append((i, getattr(item, "text", ""), getattr(item, "text", "")))
    return results

def refine_docling_table_cells_with_pdfplumber(
    pdf_path: str,
    tables: List[Any],
) -> List[Tuple[int, int, str, str]]:
    """
    Duyệt toàn bộ tables, refine text của từng cell dùng pdfplumber theo bbox.
    Trả về list (table_idx, cell_idx, old_text, refined_text).
    """
    results = []

    with pdfplumber.open(pdf_path) as pdf:
        for table_idx, table in enumerate(tables):
            try:

                if not getattr(table, "prov", None):
                    continue
                prov = None
                for p in table.prov:
                    if getattr(p, "page_no", None):
                        prov = p
                        break
                if prov is None:
                    continue
                page_no = int(prov.page_no)

                if hasattr(table, '_data'):
                    table_data = table._data
                elif hasattr(table, 'data'):
                    table_data = table.data
                else:
                    continue
                cells = getattr(table_data, 'table_cells', [])
                if not cells:
                    continue

                page = pdf.pages[page_no - 1]
                H = float(page.height)

                if getattr(page, "rotation", 0) in (90, 180, 270):
                    page = page.rotate(360 - page.rotation)
                    H = float(page.height)
                for cell_idx, cell in enumerate(cells):
                    try:
                        if not getattr(cell, "bbox", None):
                            continue
                        bbox = cell.bbox
                        l, t, r, b = float(bbox.l), float(bbox.t), float(bbox.r), float(bbox.b)

                        origin = str(getattr(bbox, "coord_origin", ""))
                        if "BOTTOMLEFT" in origin:
                            plumber_bbox = (l, H - t, r, H - b)
                        elif "TOPLEFT" in origin:
                            plumber_bbox = (l, t, r, b)
                        else:
                            continue

                        cropped = page.crop(plumber_bbox)
                        chars = cropped.chars
                        if not chars:
                            results.append((table_idx, cell_idx, getattr(cell, "text", ""), getattr(cell, "text", "")))
                            continue
                        refined = _reconstruct_text_from_chars(chars)
                        old_text = getattr(cell, "text", "")
                        results.append((table_idx, cell_idx, old_text, refined))
                    except Exception as e:
                        results.append((table_idx, cell_idx, getattr(cell, "text", ""), getattr(cell, "text", "")))
            except Exception as e:
                pass  
    return results

# def process_pdf_to_markdown(input_path: str, output_dir: str) -> None:
#     """
#     Process a PDF file using Docling, refine texts and tables with pdfplumber,
#     export to Markdown, and save to the specified output directory.
#     """

#     os.makedirs(output_dir, exist_ok=True)

#     converter = DocumentConverter()
#     result = converter.convert(input_path)

#     texts = result.document.texts
#     fixed_texts = refine_docling_texts_with_pdfplumber(
#         pdf_path=input_path,
#         docling_text_items=texts
#     )

#     for idx, old, new in fixed_texts:
#         if new and new != old:
#             try:
#                 texts[idx].text = new
#             except Exception:
#                 pass

#     tables = result.document.tables
#     fixed_tables = refine_docling_table_cells_with_pdfplumber(
#         pdf_path=input_path,
#         tables=tables
#     )

#     for table_idx, cell_idx, old, new in fixed_tables:
#         if new and new != old:
#             try:
#                 table = tables[table_idx]
#                 if hasattr(table, '_data'):
#                     table._data.table_cells[cell_idx].text = new
#                 elif hasattr(table, 'data'):
#                     table.data.table_cells[cell_idx].text = new
#             except Exception:
#                 pass

#     markdown = result.document.export_to_markdown()

#     output_file = os.path.join(output_dir, os.path.basename(input_path) + ".md")
#     with open(output_file, "w", encoding="utf-8") as f:
#         f.write(markdown)
#     print(f"Markdown saved to {output_file}")
    

# if __name__ == "__main__":
#     file_paths = glob("input_test_data_final/*.pdf") 
#     output_directory = "output_markdown_final"
#     for input_pdf in tqdm(file_paths, desc="Processing PDFs"):
#         if os.path.exists(os.path.join(output_directory, os.path.basename(input_pdf) + ".md")):
#             continue
#         process_pdf_to_markdown(input_pdf, output_directory)



In [11]:
output_dir = "output_markdown_final"
input_path = "input_test_data_final/Public_668.pdf"

os.makedirs(output_dir, exist_ok=True)

pipeline_options = PdfPipelineOptions()
pipeline_options.images_scale = 2.0                # increase resolution
pipeline_options.generate_page_images = True
pipeline_options.generate_picture_images = True
converter = DocumentConverter(
    format_options = {
      InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)
result = converter.convert(input_path)

texts = result.document.texts
fixed_texts = refine_docling_texts_with_pdfplumber(
    pdf_path=input_path,
    docling_text_items=texts
)

for idx, old, new in fixed_texts:
    if new and new != old:
        try:
            texts[idx].text = new
        except Exception:
            pass

tables = result.document.tables
fixed_tables = refine_docling_table_cells_with_pdfplumber(
    pdf_path=input_path,
    tables=tables
)

for table_idx, cell_idx, old, new in fixed_tables:
    if new and new != old:
        try:
            table = tables[table_idx]
            if hasattr(table, '_data'):
                table._data.table_cells[cell_idx].text = new
            elif hasattr(table, 'data'):
                table.data.table_cells[cell_idx].text = new
        except Exception:
            pass

for element, _ in result.document.iterate_items():
    if isinstance(element, PictureItem):
        img = element.get_image(result.document)
        img.save(f"temp.png", "PNG")

markdown = result.document.export_to_markdown()

output_file = os.path.join(output_dir, os.path.basename(input_path) + ".md")
with open(output_file, "w", encoding="utf-8") as f:
    f.write(markdown)
print(f"Markdown saved to {output_file}")

2025-11-11 10:53:35,613 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-11 10:53:35,615 - INFO - Going to convert document batch...
2025-11-11 10:53:35,616 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 02e213d66fe10d5cd7525796b8c0a9af
2025-11-11 10:53:35,616 - INFO - Auto OCR model selected ocrmac.
2025-11-11 10:53:35,616 - INFO - Accelerator device: 'mps'
2025-11-11 10:53:36,739 - INFO - Accelerator device: 'mps'
2025-11-11 10:53:36,909 - INFO - Processing document Public_668.pdf
2025-11-11 10:53:40,497 - INFO - Finished converting document Public_668.pdf in 4.88 sec.


Markdown saved to output_markdown_final/Public_668.pdf.md


In [6]:
result.document

DoclingDocument(schema_name='DoclingDocument', version='1.8.0', name='Public_668', origin=DocumentOrigin(mimetype='application/pdf', binary_hash=13594526954669064370, filename='Public_668.pdf', uri=None), furniture=GroupItem(self_ref='#/furniture', parent=None, children=[], content_layer=<ContentLayer.FURNITURE: 'furniture'>, meta=None, name='_root_', label=<GroupLabel.UNSPECIFIED: 'unspecified'>), body=GroupItem(self_ref='#/body', parent=None, children=[RefItem(cref='#/pictures/0'), RefItem(cref='#/texts/0'), RefItem(cref='#/texts/1'), RefItem(cref='#/texts/2'), RefItem(cref='#/groups/0'), RefItem(cref='#/texts/5'), RefItem(cref='#/texts/6'), RefItem(cref='#/pictures/1'), RefItem(cref='#/texts/7'), RefItem(cref='#/texts/8'), RefItem(cref='#/pictures/2'), RefItem(cref='#/texts/9'), RefItem(cref='#/pictures/3'), RefItem(cref='#/pictures/4'), RefItem(cref='#/groups/1'), RefItem(cref='#/texts/12'), RefItem(cref='#/texts/13'), RefItem(cref='#/texts/14'), RefItem(cref='#/texts/15'), RefItem

In [13]:
from glob import glob

print(len(glob("input_test_data_final/*.pdf")))
print(len(glob("output_markdown_final/*.md")))

58
58


In [22]:
import json
with open("data/processed/summary/private_test_data_summary.json", "r") as f:
    data = json.load(f)

for d in data:
    if not d["document_summary"]:
        d["document_summary"] = d["chunks"][0]["chunk_summary"]

with open("data/processed/summary/private_test_data_summary.json", "w") as f:
    json.dump(data, f, ensure_ascii=False)
print(len(data))


58


In [21]:
for x in data:
    if not x["document_summary"]:
        a = x

a

{'file_path': 'data/processed/private_test_data_md/Public_642.pdf.md',
 'document_content': '<!-- image -->\n\n## VIETTEL AI RACE\n\n## GIÁ VẬT LIỆU XÂY DỰNG THÁNG 4 NĂM 2025 TRÊN ĐỊA BÀN TỈNH LẠNG SƠN: HỆ THỐNG CỬA ĐI, CỬA SỔ, VÁCH KÍNH,\n\nTD642\n\nLần ban hành: 1\n\n| Stt   | Nhóm vật liệu                                                                                                             | Tên vật liệu, loại vật liệu                                                                                               | Đơn vị tính                                                                                                               | Tiêu chuẩn kỹ thuật                                                                                                       | Quy cách                                                                                                                  | Nhà sản xuất                                                                                        

In [12]:
import json
with open("data/processed/summary/private_test_data_summary.json", "r") as f:
    data = json.load(f)

# for d in data:
#     if not d["document_summary"]:
#         d["document_summary"] = d["chunks"][0]["chunk_summary"]
#     d["document_summary"] = f"Tóm tắt cho tài liệu {d['file_path'].split('/')[-1]}." + " " + d["document_summary"]

# with open("data/processed/summary/private_test_data_summary.json", "w") as f:
#     json.dump(data, f, ensure_ascii=False)
# print(len(data))

In [13]:
data[0]

{'file_path': 'data/processed/private_test_data_md/Public_675.pdf.md',
 'document_content': '•F\n\nTTTITTTITTTIT\n\n<!-- image -->\n\n| VIETTEL AI RACE VẼ CÁC KÝ HIỆU, QUY ƯỚC DÙNG TRONG BẢN VẼ ĐIỆN: VÁCH NGĂN VÀ BỘ PHẬN   | TD675 Lần ban hành: 1   |\n|----------------------------------------------------------------------------------------|-------------------------|\n\n## 1. Ký hiệu vách ngăn\n\nCác ký hiệu trong Điều này được quy ước để thể hiện các loại vách ngăn trên mặt bằng trong các bản vẽ có tỷ lệ 1:200 và nhỏ hơn. Ký hiệu thể hiện bằng nét liền đậm (kèm theo chú thích về vật liệu). Trường hợp bản vẽ tỷ lệ 1:50 và lớn hơn, ký hiệu vách ngăn phải thễ hiện chi tiết vật liệu và cấu tạo theo đúng tỷ lệ tính toán của kết cấu.\n\n| Tên ký hiệu                                                      | Ký hiệu   | Chú thích                                                                                                                                                      |\n|---------------

In [15]:
import pandas as pd
question = pd.read_csv("data/processed/private_question/question.csv")
question

,Question,A,B,C,D
0,Tuổi thọ trung bình của tấm pin năng lượng mặt...,20 – 25 năm,10 – 12 năm,5 – 7 năm,10 - 15 năm
1,Biểu tượng nào hiển thị màu lục khi động cơ se...,Biểu tượng chế độ thủ công,Biểu tượng gia nhiệt khoang chứa,Biểu tượng hoạt động của động cơ servo,Biểu tượng báo động
2,"Theo tài liệu Public_656, LED nào sáng khi trố...",Toner LED,Drum LED,Error LED,Ready LED
3,Nút bàn di chuột phải có chức năng gì?,Hoạt động như nút trái chuột ngoài,Hoạt động như nút phải chuột ngoài,Dùng để cuộn trang,Dùng để bật/tắt touchpad
4,Thành phần nào dùng để đặt giới hạn tối đa có ...,Manual Settings Limits,Auto Settings Limits,Mechanical Settings Limits,Servo Movement Alarms
...,...,...,...,...,...
277,"Trong TD635, có tổng cộng bao nhiêu loại đầu d...",3,4,5,6
278,Nếu một thợ kim hoàn cần tạo khuôn và thiết kế...,"Có, vì máy cho độ chi tiết cực cao và phù hợp ...","Không, vì chỉ dùng cho giáo dục","Không, vì chỉ in được mô hình kích thước lớn","Có, vì phù hợp với các sản phẩm đơn giản"
279,“Bu lông” và “Cupler” trong TD647 có Tiêu chuẩ...,TCVN 197-1:2014 và TCVN 1651:2018,ASTM A370 và ASTM A53,TCVN 1916:1995 và TCVN 8163:2009,ISO 15630-1:2010 và TCVN 7937-1:2013
280,Với hóa đơn điện hàng tháng khoảng 3 triệu đồn...,Hệ điện mặt trời hòa lưới công suất 5 kWp,Hệ điện mặt trời 3 kWp,Hệ điện mặt trời 1 kWp,Cả 3 đáp án trên
